# Описание

Ноутбук по проекту Ford vs Ferrari

Целью проекта является создание нейросети, способной определять по предоставленной картинке, к какому из классов относится представленное авто. 
В процессе будет совершенстоваться навык программирования, а также появится понимание задач компьютерного зрения в реальной жизни.

# 0. Импорт библиотек

Первым делом проверим, подключена ли видеокарта:

In [1]:
!nvidia-smi - L

Карта подключена. Теперь подключим библиотеку ImageDataAugmentor:

In [2]:
!pip install git+https: // github.com/mjkvaak/ImageDataAugmentor update tensorflow

Теперь подгрузим основные библиотеки:

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import zipfile
import csv
import sys
import os
import itertools
# Подгрузим все необходимые нам инструменты из tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import ResNet101V2, EfficientNetB3
from ImageDataAugmentor.image_data_augmentor import *
import albumentations as a

from PIL import Image

Посмотрим версии используемых библиотек:

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

## 0.1. Настройка параметров

Зафиксируем настройки:

In [5]:
!pip freeze > requirements.txt

Настроим основые параметры, которые мы будем использовать в дальнейшем

In [6]:
EPOCHS = 8  # кол-во эпох для обучения
BATCH_SIZE = 16  # уменьшаем batch если сеть большая, иначе не влезет в память на GPU
LR = 1e-3  # шаг обучения
VAL_SPLIT = 0.2  # кол-во данных для валидации(20%)

CLASS_NUM = 10  # количество выходных классов
H_IMG_SIZE = 150  # Горизонтальный размер изображения
V_IMG_SIZE = 150  # Вертикальный размер изображения
IMG_CHANNELS = 3   # кол-во каналов для rgb-изображения
# Параметры, до которых мы будем преобразовывать изображение
input_shape = (V_IMG_SIZE, H_IMG_SIZE, IMG_CHANNELS)

DATA_PATH = '../input/sf-dl-car-classification/'  # Директория с данными
PATH = "../working/car/"  # рабочая директория
RANDOM_SEED = 42

# 1. Обзор исходных данных

Распакуем картинки, с которыми предстоит работать в данном ноутбуке:

In [7]:
print('Разархивируем изображения')
# Распаковка архивов с данными
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/sf-dl-car-classification/"+data_zip, "r") as z:
        z.extractall(PATH)

print(os.listdir(PATH))

Загрузим csv файлы для дальнейшей работы:

In [8]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")

Посмотрим шапку тренировочной выборки. Здесь всего два столбца: id с названием фото и его расширением и класс, к которому относится это фото

In [9]:
train_df.head()

Посмотрим пример сабмита: выглядит аналогично тренировочной выборке.

In [10]:
sample_submission.head()

# 2. EDA (Анализ исходных данных)

Для начала необходимо посмотреть сколько всего классов данных мы имеем:

In [11]:
train_df.Category.value_counts()

Проверим гистограмму распределения классов:

In [12]:
train_df.Category.hist()

Как видно из гистограммы, данные распределены более-менее равномерно по классам. Проверим как выглядят фотографии и их размер:

In [13]:
# выводим случайные 9 изображений с обозначением классов и указанием размера по осям
plt.figure(figsize=(12, 8))
random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values
for index, path in enumerate(random_image_paths):
    im = Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
plt.show()

Из демонстрации видно, что изображения в выборке разного размера. Также присутствует различие в контрастности и различается ракурс, с которого представлены автомобили. Так как изображений в трейне не очень много, лучше использовать аугментации для увеличения обучающей выборки.

# 3. Исследование моделей нейросети

В качестве исследуемых моделей выбраны Xception, InceptionV3, ResNet101V2 и EfficientNetB3. Все выборы сделаны исходя и Leaderboard'а, представленного на сайте https://paperswithcode.com/. Помимо прочего, я старался выбрать модели, которые не уступают по показателям на ImageNet, но при которых время обучения модели не будет критически большим.

* Показатели по всем исследуемым моделям находятся в пределах 80+% по точности предсказаний.

* На первом этапе исследования сетей будет использован способ аугментации с помощью ImageDataGenerator. 

* Используется перенос обучения с существующих сетей на новые, которые мы моделируем.

* Оптимизатором шага обучения я выбрал Adamax, частный вариант Adam. Эти два алгоритма являются более чувствутельными к весам исследуемой модели. Adamax показал себя лучше в вычислениях на нижеприведенных моделях, поэтому и стал финальным используемым оптимизатором.

* В качестве Loss-функции выбрана категориальная кросс-энтропия


## 3.1. Модели Xception, InceptionV3 и ResNet101V2

Данные модели не  могут получить на вход изображения в том виде, в котором они хранятся в выборках. Поэтому, для начала необходимо их нормализовать с помощью rescale. 

Начнем с создания генератора для аугментации изображений и применим его.

### 3.1.1. Аугментация данных для моделей, требующих rescale

Установим параметры аугментации изображений нашего генератора ImageDataGenerator :

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,  # т.к мы используем модели без встроенного rescale
    rotation_range=5,  # диапазон поворота
    width_shift_range=0.1,  # диапазон растягивания по ширине
    height_shift_range=0.1,  # диапазон растягивания по высоте
    horizontal_flip=False,  # переворот относительно горизонтали
    shear_range=0.1,  # диапазон сдвига
    zoom_range=0.1,  # диапазон увеличения
    fill_mode='reflect',  # заполнение края отзеркаливанием изображения
    validation_split=VAL_SPLIT)  # указываем разбиение на train/val

Теперь запустим ImageDataGenerator и проведем аугментацию изображений для обучающей и валидационной выборок:

In [15]:
# Аугментации обучающей выборки
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с изображениями
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # указана тренировочная выборка
# Аугментации валидационной выборки
val_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # указана валидационная выборка

Как мы видим, в результате аугментации у нас получилось 15561 изображение, из которых 12452 изображения в обучающей выборке, и еще 3109 в валидационной.

Просмотрим результаты аугментации с выведением размеров по осям на примере 6 изображений из обучающей выборки:

In [16]:
x, y = train_generator.next()
print('Пример изображений из train_generator')
plt.figure(figsize=(12, 8))

for i in range(0, 6):
    image = x[i]
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
plt.show()

Видно, что все изображения приведены к одному размеру, а также наблюдаются прочие аугментации, заполнение края отзеркаливанием изображения. 
Реализуем демонстрацию изображений из валидационной выборки аналогичным образом:

In [17]:
x, y = val_generator.next()
print('Пример изображений из val_generator')
plt.figure(figsize=(12, 8))

for i in range(0, 6):
    image = x[i]
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
plt.show()

На валидационной выборке мы получили такие же  преобразования, как и на обучающей.

Теперь, когда проведена аугментация изображений для первых трех исследуемых моделей - можно приступать к их созданию.

### 3.1.2. Создание head

Так как мы пытаемся исследовать все модели в одинаковых условиях, создадим для них единую "голову", которую будем использовать в этом пункте:
* Слой GlobalMaxPool2D, который используется для объединения Base-model и head
* Dropout-слой - Направлен на уменьшение переобучения сети засчет "выключения" нейронов из процесса обучения с определенной вероятностью
* Выходной слой на 10 классов с функцией активации softmax

In [51]:
head = Sequential([
    GlobalMaxPool2D(),
    Dropout(0.3),
    Dense(CLASS_NUM, activation='softmax')
])

### 3.1.3.  Модель Xception

Приступим к созданию и обучению модели: 

На данном этапе исследуется модель Xception. 
Загрузим базовую часть модели с весами imagenet, то есть используем перенос обучения с уже существующей сети на новую, которую мы создаем:

In [19]:
# Загружаем базовую модель с весами imagenet
base_model = Xception(weights='imagenet',
                      include_top=False, input_shape=input_shape)
# Указываем, что модель может обучаться
base_model.trainable = True

Теперь соединим нашу базовую модель и head из п.п.3.1.2 в единую модель:

In [20]:
model = Sequential([
    base_model,
    head
])

Проведем компиляцию модели, указав способ определения Loss-функции, используемый оптимизатор,шаг обучения и исследуемую метрику:

In [21]:
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Посмотрим количество слоев в базовой модели:

In [22]:
print(len(base_model.layers))

На данном этапе мы обучаем все слои, то есть веса в них могут перезаписываться.

Теперь создадим список callbacks, которые мы будем использовать во время обучения модели:
* checkpoint - сохраняет после каждой эпохи модель в файл в случае, если валидационная точность модели повышается.
* earlystop - Прерывает обучение модели в случае, если на протяжении patience эпох не наблюдается улучшения val_accuracy

In [23]:
# Указываем параметры наших коллбэков
checkpoint = ModelCheckpoint('best_model.hdf5', monitor=[
                             'val_accuracy'], verbose=1, mode='max')
earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=5, restore_best_weights=True)
# Создаем список с используемыми коллбэками
callbacks_list = [checkpoint, earlystop]

Обучим модель, созданную на основе Xception, с помощью аугментированных данных, а также проведем проверку точности на валидационных данных:

In [24]:
# Приступаем к обучению нашей модели
xcept_fit = model.fit(
    train_generator,  # Используемые данные для обучения
    # Количество итераций обучения за одну эпоху
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,  # Используемые данные для валидации
    # количество итераций валидации за одну эпоху
    validation_steps=len(val_generator),
    epochs=EPOCHS,  # количество эпох
    callbacks=callbacks_list  # Используемые коллбэки
)

Сохраним результаты в рабочую директорию и загрузим лучшие веса модели:

In [25]:
model.save('../working/model_xcept_1.hdf5')
model.load_weights('best_model.hdf5')

Проверим точность, которая получилась после обучения используя model.evaluate_generator (предсказывает сначала при помощи входных данных, а затем оценивает точность и сравнивает ее с результатом, полученным на валидации):

In [26]:
scores = model.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики потерь и точности предсказаний на тренировончной и валидационной выборках:

In [27]:
acc = xcept_fit.history['accuracy']
val_acc = xcept_fit.history['val_accuracy']
loss = xcept_fit.history['loss']
val_loss = xcept_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Мы получили достаточно хорошие метрики, хотя похоже, что модель начала переобучаться. На графике точности четко виден пик на 4 эпохе, после которого начинается ее снижение и образуется низина вплоть до 7 эпохи, на которой снова происходит рост метрики.

С графиками функции потерь ситуация похожа. До 4 эпохи идет спад функции, однако после этого потери снова начинают расти и возвращаются к результату 4 эпохи только на 8.

При этом точность на валидационной выборке мы получили довольно неплохую: 90.99%

### 3.1.4. Модель inception_v3

В данном подпункте мы будем проводить исследование, аналогичное пп.3.1.3., но в качестве базовой модели будем использовать Inception_v3. 

Начнем с загрузки базовой части модели с весами ImageNet. Возможность обучения установим = True. Это значит, что веса, которыми уже обладает базовая модель, могут перезаписываться в результате обучения.

In [28]:
# Загрузим базовую часть модели InceptionV3
base_model = InceptionV3(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Так же, как и в предыдущем пункте, соберем воедино части нашей модели:

In [29]:
model_incept = Sequential([
    base_model,
    head
])

Проведем компиляцию модели:

In [30]:
model_incept.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Теперь обучим нашу модель на аугментированных данных, полученных в train_generator, и проведем проверку точности на валидацонных данных:

In [31]:
inceptionv3_fit = model_incept.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним нашу модель и загрузим лучшие веса:

In [32]:
model_incept.save('../working/model_incept_1.hdf5')
model_incept.load_weights('best_model.hdf5')

Как и ранее, проведем проверку точности при помощи evaluate_generator:

In [33]:
scores = model_incept.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики потерь и точности предсказаний на тренировончной и валидационной выборках:

In [34]:
acc = inceptionv3_fit.history['accuracy']
val_acc = inceptionv3_fit.history['val_accuracy']
loss = inceptionv3_fit.history['loss']
val_loss = inceptionv3_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

На графиках видны характерные пики и провалы при обучении сети. Так, на второй эпохе был сильный рост функции потерь, который быстро снизился. Аналогичная ситуация наблюдается на шестой эпохе. После седьмой эпохи функция потерь снова немного возрастает. 

С графиками точности происходит примерно то же самое, на обучающей выборке точность постепенно растет, без спадов, а на валидации мы получаем высокую точность на шестой эпохе, после которой следует снижение и, к восьмой эпохе, возвращение примерно в то же значение, которое получено две эпохи назад. Возможно,если увеличить количество эпох - станет более явно видно, переобучилась сеть или нет, на данном этапе однозначный вывод сделать сложно.

### 3.1.5. Модель ResNet101V2

В данном подпункте проведем исследование модели ResNet101V2. Все этапы ничем не отличаются от проделанных в пп.3.1.3-3.1.4:

In [35]:
# Загрузим базовую часть модели с весами ImageNet
base_model = ResNet101V2(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Соберем воедино части нашей исследуемой модели:

In [36]:
# Собираем модель
model_resnet = Sequential([
    base_model,
    head
])

Проведем компиляцию модели:

In [37]:
model_resnet.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Теперь приступим к обучению нашей модели ResNet101v2:

In [38]:
resnet101v2_fit = model_resnet.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним полученную модель и загрузим ее лучшие веса:

In [39]:
model_resnet.save('../working/model_resnet_1.hdf5')
model_resnet.load_weights('best_model.hdf5')

Проверим точность нашей модели с помощью evaluate_generator:

In [40]:
scores = model_resnet.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики потерь и точности предсказаний на тренировончной и валидационной выборках:

In [41]:
acc = resnet101v2_fit.history['accuracy']
val_acc = resnet101v2_fit.history['val_accuracy']
loss = resnet101v2_fit.history['loss']
val_loss = resnet101v2_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

На графиках функции потерь и точности видно, что модель в промежутке после 4 и до 7 эпохи имела плохие показатели метрик, хуже чем на самой 4 эпохе. К 7 эпохе показатели снова улучшились, однако затем произошел очередной спад точности и рост функции потерь. Можно сделать вывод, что модель начала переобучаться.

## 3.2. Модель EfficientNetB3

Модель EfficientNetB3 немного отличается от остальных тем, что для нее преобразование с помощью rescale не нужно. Это связано с тем, что она принимает на вход тензоры пикселей с диапазоном значений от 0 до 255. Предыдущие модели принимали диапазон от -1 до 1, поэтому для них проводились необходимые преобразования.

Для дальнейшей работы необходимо переделать ImageDataGenerator, убрав из него rescale. Остальные параметры остаются без изменений.

### 3.2.1. Аугментация данных

Зафиксируем настройки для нашего генератора, исключив ис перечня rescale=1./255. Описание каждой строчки, что она делает можно найти выше, при первой аугментации:

In [46]:
train_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=False,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='reflect',
    validation_split=VAL_SPLIT)  # set validation split

test_datagen = ImageDataGenerator()

In [47]:
train_generator.reset()
val_generator.reset()

Проведем аугментацию изображений, используя наши настройки ImageDataGenerator'а:

In [48]:
train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # Обучающая выборка

val_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # Валидационная выборка

Теперь, как и в предыдущих пунктах, загрузим базовую часть модели с весами ImageNet и укажем, что она может обучаться:

In [49]:
# Загружаем базовую часть модели с указанием trainable= True
base_model = EfficientNetB3(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Ниже я переобъявлю head, иначе по какой-то причине при выполнении выскакивает ошибка:

In [53]:
head = Sequential([
    GlobalMaxPool2D(),
    Dropout(0.3),
    Dense(CLASS_NUM, activation='softmax')
])

Теперь собираем в единую модель наши base_model и head:

In [54]:
model_efficientnetb3 = Sequential([
    base_model,
    head
])

Проведем компиляцию модели с указанными параметрами:

In [55]:
model_efficientnetb3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Наконец, запустим обучение созданной нами модели с демонстрацией точности на обучающей и валидационной выборках:

In [56]:
efficientnetb3_fit = model_efficientnetb3.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним полученную модель и загрузим ее лучшие веса:

In [57]:
model_efficientnetb3.save('../working/model_efficientnetb3_1.hdf5')
model_efficientnetb3.load_weights('best_model.hdf5')

Проверим полученную точность (в %) на валидационной выборке с помощью evaluate_generator:

In [58]:
scores = model_efficientnetb3.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики потерь и точности предсказаний на тренировончной и валидационной выборках:

In [59]:
acc = efficientnetb3_fit.history['accuracy']
val_acc = efficientnetb3_fit.history['val_accuracy']
loss = efficientnetb3_fit.history['loss']
val_loss = efficientnetb3_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Как мы можем увидеть - результат полученный при использовании EfficientNetB3 получился наиболее высоким из всех. В дальнейшем я планирую использовать ее для финального сабмита. Но для начала, я планирую попробовать улучшить модели, изменив способ аугментации и собрав для них новую голову.

# 4. Улучшение моделей

Для улучшения исследуемых моделей попробуем воспользоваться другим аугментором (ImageDataAugmentor), использующим albuminations. Помимо этого попробуем вместо фиксированного learning rate использовать ExponentialDecay, который реализует его затухание.

    На этом этапе было очень много различных экспериментов, в результате которых я выработал всю квоту на использование GPU. Это связано с постоянными перерасчетами, например при изменении структуры шапки, шага обучения, оптимизатора и количества параметров при аугментации. Я не берусь сказать, что выбранные в итоге параметры являются лучшими, вероятно это совсем не так, но они оказали хоть какое-то положительное влияние на обучение модели, так как в начале результаты получались в среднем на 3-4% хуже чем при исследовании моделей в п.3. 
    Среди прочего - были рассмотрены варианты с фиксированным learning rate, но exponentialDecay показал лучшие результаты на первых трех моделях, были рассмотрены различные варианты функции активации для dense-слоя: Elu и relu (функция relu показала чуть лучший результат, поэтому в финальной версии используется она).
    Для albuminations были рассмотрены различные параметры аугментации, в процессе постоянно менялись их вероятности. Так, при наличии настроек яркости a.RandomBrightness - качество обучения падало на 2%. В финальной версии этого параметра нет.
    Два сабмита, реализованные на базе модели EfficientNetB3, различаются между собой настройками ImageDataAugmentor: в первом случае используются настройки из п.п.4.1., а во втором используются они же, но без параметров a.RandomBrightnessContrast.

## 4.1. Аугментация

Пропишем парметры аугментации ImageDataAugmentor, используя albuminations:
* Установим размытие по Гауссу
* Установим Гауссовский шум
* Установим параметры сдвига, масштабирования и поворота изображения
* Установим яркость и контрастность изображения

In [60]:
augment_module = a.Compose([
    # добавляем размытие по Гауссу и шум с вероятностью 7%
    a.Blur(p=0.07),
    a.GaussNoise(p=0.07),
    #  Установим параметры сдвига,поворота и масштабирования, а также укажем их вероятность.
    a.ShiftScaleRotate(shift_limit=0.08,
                       scale_limit=0.05,
                       border_mode=4,
                       rotate_limit=20,
                       p=0.65),

    a.HueSaturationValue(),  # случайный оттенок и насыщенность
    a.HorizontalFlip(),

    a.Resize(V_IMG_SIZE, H_IMG_SIZE),
    # установим случайную яркость и контрастность изображений с вероятностью 50%

    a.OneOf([
            a.RandomBrightnessContrast(
                brightness_limit=0.2, contrast_limit=0.2),
            a.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)],
            p=0.5)
])

Теперь, используя созданный выше блок, запишем параметры для генератора:

In [61]:
# Подготовка параметров для аугментации:
all_datagen = ImageDataAugmentor(
    augment=augment_module,
    seed=RANDOM_SEED,
    validation_split=VAL_SPLIT)

Сбросим используемые генераторы:

In [62]:
train_generator.reset()
val_generator.reset()

Теперь запустим генераторы с используемыми настройками аугментации:

In [63]:
train_generator = all_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training')

val_generator = all_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation')

Создадим блок, который будет нам демонстрировать аугментированные изображения из тренировочной и валидационной выборок:

In [64]:
def show_first_images(generator, count=6, labels=True, figsize=(20, 5), normalized=False):
    generator = itertools.islice(generator, count)
    fig, axes = plt.subplots(nrows=1, ncols=count, figsize=figsize)
    for batch, ax in zip(generator, axes):
        if labels:
            img_batch, labels_batch = batch
            # берем по одному изображению из каждого батча
            img, label = img_batch[0], np.argmax(labels_batch[0])
        else:
            img_batch = batch
            img = img_batch[0]
        if not normalized:
            img = img.astype(np.uint8)
        ax.imshow(img)
        # метод imshow принимает одно из двух:
        # - изображение в формате uint8, яркость от 0 до 255
        # - изображение в формате float, яркость от 0 до 1
        if labels:
            ax.set_title(f'Class: {label}')
    plt.show()

Просмотрим результаты:

In [65]:
# Обучающая выборка
print('Train:')
show_first_images(train_generator)

# Валидационная выборка
print('Val:')
show_first_images(val_generator)

## 4.2. Создание Head и CallBacks

Данный пункт неоднократно переделовался в силу того, что я постоянно менял структуру и количество слоев в шапке. Двумя наиболее хорошими оказались:
* Вариант с закомментированной частью кода
* Вариант, при котором закомментированная часть тоже используется

На данном этапе я пробовал различные функции активации, отсутствие\присутствие батч-нормализации, различные вероятности для Dropout-слоя и различное количество юнитов в dense-слое. Как сказано в начале самого пункта, функция активации relu показала себя немного лучше, чем elu в результате исследований, поэтому в финальной версии используется именно она.

Собираем новую head для дальнейшего исследования:

In [66]:
head = Sequential([
    GlobalMaxPool2D(),  # GlobalMaxPool2D для объединения base_model и head
    # Dense-слой с 256 нейронами и функцией активации relu
    Dense(256, activation='relu'),
    BatchNormalization(),  # Батч-нормализация
    Dropout(0.5),  # Dropout-слой с вероятностью 0.5
    # Закомментированные слои, о которых сказано выше
    #Dense(64, activation='relu'),
    # Dropout(0.25),
    # выходной слой с активацией softmax
    Dense(CLASS_NUM, activation='softmax')
])

Немного изменим коллбэки. Я решил что при 8 эпохах на обучении ставить 5 для earlystopping - это не совсем логично. Поэтому уменьшил значение хотя бы до 4 эпох без положительного прогресса в val_accuracy.

In [67]:
# Немного изменим значения в earlystopping
earlystop = EarlyStopping(monitor='val_accuracy',
                          patience=4, restore_best_weights=True)
callbacks_list = [checkpoint, earlystop]

## 4.3. Модель Xception

По аналогии с исследованиями в пункте 3, проведем обучение модели Xception, но в качестве  обучающей и валидационной выборок будем использовать данные, преобразованные с помощью ImageDataAugmentor. Также вместо старой head будем использовать новую, созданную в пп.4.2.

Начнем, как обычно, с загрузки базовой модели Xception с весами из ImageNet:

In [68]:
# Загружаем базовую часть модели с весами ImageNet, ставим возможность обучения=True
base_model = Xception(weights='imagenet',
                      include_top=False, input_shape=input_shape)
base_model.trainable = True

Чтобы сделать код более правильным, не добавлять миллиард генераторов для моделей, которые могут обойтись без rescale - я сделал генератор в единственном виде, а для моделей, которые не могут получать на выход данные без rescale, добавил lambda-слой, реализующий ту же самую функцию.

Оптимизатор используем тот же, что и в предыдущем пункте, но заменим постоянный learning rate на ExponentialDecay(при экспериментах с моделями показал результат лучше, чем фиксированный LR).

Соберем модель воедино из имеющихся у нас частей и проведем компиляцию:

In [69]:
# Соеднияем Lambda-слой, базовую модель и head в одну модель
model_xception = Sequential([
    Lambda(lambda x: x/255),  # Слой, реализующий rescale
    base_model,
    head
])
# Производим компиляцию модели. Вместо фиксированного learning rate используем ExponentialDecay
model_xception.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=ExponentialDecay(LR, decay_steps=100, decay_rate=0.95)), metrics=["accuracy"])

Обучаем полученную модель на имеющихся данных:

In [70]:
# Обучаем модель на тренировочной выборке,
# проверяем точность на валидационной и тренировочной выборках
xception_fit = model_xception.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним нашу модель и загрузим лучшие веса для нее:

In [71]:
model_xception.save('../working/model_xception1.hdf5')
model_xception.load_weights('best_model.hdf5')

Проверим полученную точность на валидационных данных с использованием evaluate_generator:

In [72]:
scores = model_xception.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики потерь и точности предсказаний на тренировончной и валидационной выборках:

In [73]:
acc = xception_fit.history['accuracy']
val_acc = xception_fit.history['val_accuracy']
loss = xception_fit.history['loss']
val_loss = xception_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Графики стали более плавными, удалось немного улучшить точность модели. Возможно, если увеличить количество эпох, модель обучится еще лучше, но это слишком долго. Но точность на обучающей выборке все же выше, чем на валидационной, и есть подозрение, что при увеличении эпох разрыв только увеличится.
Посмотрим, как поведут себя другие модели.

## 4.4. Модель Inception_v3

Модель InceptionV3 будет исследоваться образом, аналогичным пп.4.3. Для начала мы загрузим базовую часть модели InceptionV3, укажем, что она может обучаться, затем соберем единую модель из частей, которые мы имеем, и обучим полученную модель на данных, полученных с помощью train_generator.

**Примечание:** Различные варианты head проверялись для каждой модели, включая эту. Я оставил наиболее универсальный, подходящий для каждой из первых 3 моделей.

In [74]:
# Загружаем базовую модель
base_model = InceptionV3(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Сделаем единую модель из имеющихся у нас частей и проведем компиляцию:

In [75]:
# Соеднияем Lambda-слой, базовую модель и head в одну модель
model_inceptionv3 = Sequential([
    Lambda(lambda x: x/255),
    base_model,
    head
])
# Проводим компиляцию модели с указанными параметрами
model_inceptionv3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=ExponentialDecay(LR, decay_steps=100, decay_rate=0.95)), metrics=["accuracy"])

Обучим полученную модель с использованием данных обучающей выборки, полученных с помощью train_generator. Параллельно с этим будем проверять валидационную точность в течение обучения на валидационной выборке:

In [76]:
# обучаем нашу модель на тренировочной выборке и проводим валидацию в течение 8 эпох
inceptionv3_fit = model_inceptionv3.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним полученную модель и загрузим ее лучшие веса:

In [77]:
model_inceptionv3.save('../working/model_inceptionv3_1.hdf5')
model_inceptionv3.load_weights('best_model.hdf5')

Проверим валидационную точность с помощью evaluate_generator:

In [78]:
scores = model_inceptionv3.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики точности и loss-функции для обеих выборок, которые мы использовали:

In [79]:
acc = inceptionv3_fit.history['accuracy']
val_acc = inceptionv3_fit.history['val_accuracy']
loss = inceptionv3_fit.history['loss']
val_loss = inceptionv3_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Графики точности и функции потерь гораздо лучше, чем у аналога из пункта 3. Почти на 2/3 обучения точность валидационной выборки выше, чем на обучающей, что хорошо. Также, удалось незначительно улучшить саму метрику.

## 4.5. Модель ResNet101V2

Для модели ResNet101v2 принципиальных различий нет - проделаем те же самые шаги.

In [80]:
# загружаем базовую модель с весами ImageNet и указываем, что она может обучаться
base_model = ResNet101V2(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Соберем единую модель из имеющихся частей. Как и в пп.4.3 и пп 4.4. используем Lambda-слой, реализующий функцию, аналогичную rescale:

In [81]:
# Собираем все части в одну модель
model_resnet101v2 = Sequential([
    Lambda(lambda x: x/255),
    base_model,
    head
])
# Проводим компиляцию модели с указанными парамтерами
model_resnet101v2.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=ExponentialDecay(LR, decay_steps=100, decay_rate=0.95)), metrics=["accuracy"])

Проведем обучение модели аналогично предыдущим подпунктам:

In [82]:
# Обучаем модель на тренировочной выборке и проверим ее точность на валидационной выборке:
resnet101v2_fit = model_resnet101v2.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)

Сохраним нашу модель и загрузим лучшие веса:

In [83]:
model_resnet101v2.save('../working/model_resnet101v2_1.hdf5')
model_resnet101v2.load_weights('best_model.hdf5')

Проверим точность на валидационной выборке с использованием evaluate_generator:

In [84]:
scores = model_resnet101v2.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Построим графики точности и функции потерь для обеих выборок:

In [85]:
acc = inceptionv3_fit.history['accuracy']
val_acc = inceptionv3_fit.history['val_accuracy']
loss = inceptionv3_fit.history['loss']
val_loss = inceptionv3_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Как и с предыдущими моделями, графики стали гораздо лучше и плавнее. Валидационные кривые начинают показывать результаты хуже обучающих только к концу обучения. К тому же удалось значительно увеличить точность предсказаний на валидационной выборке - на 15.41%, это очень сильное изменение.

## 4.5. Модель EfficientNetB3 

С моделью EfficientNetB3 получилась отдельная история - как ни пытался, ее результаты получались в среднем процента на 4 меньше, чем в пункте 3. Собственно, это и побудило попробовать разные варианты head, разные настройки для ImageDataAugmentor, попробовать экспоненциальное затухание вместо фиксированного learning rate и даже попробовать регуляризацию функции активации и bias-коэффициентов. Также именно тут я попробовал первый раз использовать Batch-нормализацию для увеличения сходимости. Такая же нормализация используется в начале пункта 4 для создания head.

Но старая шапка давала все еще давала метрики хуже, чем в пункте 3, поэтому я решил попробовать сделать другую, так появилась head_new. У нее, как и у обычной head, два варианта реализации, показывающие более-менее одинаковые метрики:
1. с закомментированным первым Dense-слоем и используемым вторым Dense-слоем
2. с закомментированным вторым Dense-слоем и используемым первым Dense-слоем

Я оставил первый вариант, мне он показался чуть лучше. Необходимо отметить, что глобальный пуллинг с использованием среднего значения тоже дал результаты лучше, чем max-пуллинг, поэтому в финальной версии я использую его.

Вариант с использованием экспоненциального затухания по итогу дал не самые лучшие результаты при EfficientNetB3, однако хорошо усвоился предыдущими тремя сетями. Поэтому здесь я решил использовать ReduceLROnPlateau. Данный способ позволяет уменьшить learning rate в случае, если нет положительного прогресса исследуемой метрики в течение **patience** эпох. Скорость обучения при этом изменияется по принципу **lr_new=lr*factor**.

С помощью сети, основанной на EfficientNetB3, я сделал первые два сабмита. Точность в них различается на ~0.5%. Поэтому я оставил только вариант, где в ImageDataAugmentor используются все  настройки из пп.4.1. Вторым вариантом было не использовать настройки a.RandomBrightnessContrast, он и дал чуть более хороший результат, но при этом переобучение наступало гораздо раньше, чем в первом случае.

Ниже представлены этапы создания модели.

Создадим новую head:

In [86]:
# Создаем head_new, которую будем использовать в дальнейшем для создания модели
head_new = Sequential([
    GlobalAveragePooling2D(),  # Используем пуллинг со средним значением
    #Dense(256, activation='relu',bias_regularizer=l2(1e-3),activity_regularizer=l2(1e-4)),
    BatchNormalization(),  # Используем batch-нормализацию
    Dropout(0.25),   # Dropout- слой с p=0.25
    Dense(256, activation='relu', bias_regularizer=l2(1e-3),
          activity_regularizer=l2(1e-4)),  # Dense-слой с l2 регуляризацией
    BatchNormalization(),  # Batch- нормализация
    # выходной слой на 10 классов и функцией активации softmax
    Dense(CLASS_NUM, activation='softmax')
])

В случае, если использовать закомментированный слой вместо текущего Dense- слоя, метрики получаются немного хуже.

Обновим Callback, добавив в него уменьшение learning rate в случае, если нет улучшения метрики в течение двух эпох

In [87]:
# Объявим новый коллбэк, который будет уменьшать скорость обучения в случае, если функция потерь не уменьшается в течение 2 эпох
lr_scheduler = ReduceLROnPlateau(monitor='val_loss',
                                 factor=0.5,  # уменьшим lr в 2 раза
                                 patience=2,  # если нет улучшения через 2 эпохи - уменьшить lr

                                 min_lr=0.0000001,  # минимальная скорость обучения
                                 verbose=1,  # выводить сообщения об уменьшении скорости
                                 mode='auto')  # выбранный способ отслеживания метрики

# создадим новый список коллбэков, который будем использовать
callbacks_list1 = [checkpoint, earlystop, lr_scheduler]

Загрузим базовую часть модели EfficientNetB3 с весами ImageNet:

In [88]:
# Загрузим базовую модель и укажем, что она может обучаться
base_model = EfficientNetB3(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = True

Соберем в единую сеть все имеющиеся у нас части:

**Примечание :** В данном случае Lambda-слой не нужен, так как модель получает на вход тензоры пикселей со значениями в диапазоне от 0 до 255.

In [89]:
# Соберем нашу исследуемую модель, используя head_new:
model_efficientnetb3 = Sequential([
    base_model,
    head_new
])
# Проведем компиляцию модели с приведенными параметрами
model_efficientnetb3.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Теперь проведем обучение нашей модели, используя новый список коллбэков (callbacks_list1):

In [90]:
# Проведем обучение модели с использованием тренировочной выборки и посмотрим метрики на валидационной выборке:
efficientnetb3_fit = model_efficientnetb3.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list1  # Новый список коллбэков
)

Сохраним нашу модель и загрузим лучшие веса:

In [91]:
model_efficientnetb3.save('../working/model_efficientnetb3_1.hdf5')
model_efficientnetb3.load_weights('best_model.hdf5')

Посмотрим точность на валидационной выборке с помощью evaluate_generator:

In [92]:
scores = model_efficientnetb3.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики точности и функции потерь на обучающей и валидационной выборках:

In [93]:
acc = efficientnetb3_fit.history['accuracy']
val_acc = efficientnetb3_fit.history['val_accuracy']
loss = efficientnetb3_fit.history['loss']
val_loss = efficientnetb3_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Сделаем Submit. Для этого, сначала сделаем генератор для тестовых данных:

In [94]:
# создадим генератор для тестовых данных
test_datagen = ImageDataAugmentor()
submission_generator = test_datagen.flow_from_dataframe(dataframe=sample_submission,
                                                        directory=PATH+'test_upload/',
                                                        x_col="Id",
                                                        y_col=None,
                                                        shuffle=False,
                                                        class_mode=None,
                                                        target_size=(
                                                            V_IMG_SIZE, H_IMG_SIZE),
                                                        batch_size=BATCH_SIZE)

Сделаем предсказание на тестовых данных, используя predict_generator:

In [95]:
predictions = model_efficientnetb3.predict_generator(
    submission_generator, steps=len(submission_generator), verbose=1)
predictions = np.argmax(predictions, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

Сохраним предсказания в submission и сделаем сабмит:

In [96]:
filenames_w_dir = submission_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames_w_dir, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission_EfNetB3.csv', index=False)

Посмотрим как выглядит результат:

In [97]:
submission.head()

По итогу выполнения подпункта я получил 2 сабмита:
1. С настройками яркости и контраста
1. Без настроек яркости и контраста

Сами результаты разительно не отличаются, примерно в 0.5%. 

Здесь я оставил только финальный результат, потому что процесс выполнения документа становился довольно большим. Второй вариант не отличается ни чем, кроме самих настроек в ImageDataAugmentor.

По итогам выполнения различных моделей, я выбрал для финального сабмита именно EfficientNetB3. Она обладает наилучшими графиками исследуемой метрики.

# 5. Выводы по моделям

В результате исследования я получил различные прогнозы по четырем моделям: Xception, InceptionV3, ResNet101V2, EfficientNetB3.

Надо сказать, что все исследования проводились при размере изображения 150х150, что могло сильно повлиять на обучение моделей. Валидационная точность у моделей получилась примерно следующая:
1. Xception без ImageDataAugmentor'a:90.99%
2. Xception с ImageDataAugmentor'ом:91.06%
3. InceptionV3 без ImageDataAugmentor'a:88.29%
4. InceptionV3 с ImageDataAugmentor'ом:89.64%
5. ResNet101V2 без ImageDataAugmentor'a:74.59%
6. ResNet101V2 с ImageDataAugmentor'ом:90.02%
7. EfficientNetB3 без ImageDataAugmentor'a:91.64%
8. EfficientNetB3 с ImageDataAugmentor'ом:91.87%

Отмечу, что модель ResNet101V2 показала себя хуже всего в данной задаче, однако именно ее метрики получили ощутимый прирост в результате изменения способа аугментации. На остальных же моделях результат не настолько ощутим, как в ее случае.

Модели, полученные в пункте 3 могут обладать довольно неплохой точностью, однако их графики исследуемых метрик имеют видимые провалы на некоторых эпохах обучения, а также почти везде присутствует переобучение.

В пункте 4, где исследовались модели с другим способом аугментации, а также изменненной "головой", графики получились более плавными, немного уменьшилось переобучение и увеличилась метрика валидационной точности. В случае с EfficientNetB3, разительного изменения в точности не наблюдается, однако есть большие изменения в графиках исследуемых характеристик.

Для дальнейшего улучшения метрик, возможно, необходимо было бы еще подумать над структурой головы, а также над способами аугментации и параметрами, используемыми при ней. Возможно стоит снова рассмотреть различные комбинации скорости обучения и используемого способа оптимизации. Но так как квота не бесконечная, а за ее рамки в 37 часов я уже один раз вышел, это останется только теорией о том, как можно улучшить точность предсказаний.

Единственное, что стоит еще отметить - вероятно, при бОльшем исходном дата-сете, используемом для обучения и валидации, метрики получились бы лучше, особенно если при этом будет соблюдаться отсутствие доминирования определенного класса в выборках.

В качестве модели, используемой для финального сабмита я решил использовать EfficientNetB3. Я не могу сказать, что хорошо выбрал все параметры для этой модели, также как и для других, но именно в ее случае графики функции потерь и точности получились наилучшими: почти на всем этапе обучения точность на валидационной выборке выше, чем на обучающей, а функция потерь на валидации почти на всем обучении ниже. Перед тем как сделать сам сабмит, я попробую использовать тонкую настройку модели:
1. Обучим сеть с "замороженной" базовой частью
2. Обучим сеть снова с "замороженной" половиной слоев базовой части
3. Обучим сеть в третий раз с "замороженной" 1\4 слоев базовой части
4. Обучим сеть финальный раз с полностью размороженной базовой частью

После этого сделаем финальный сабмит.



# 6. Fine-Tuning и финальный Submission

## 6.1. Применение Fine-Tuning к модели EfficientNetB3

На данном этапе было несколько исследований модели с использованием fine-tuning'а. Один сабмит сделан без изменения размера ( дал около 95% точность на тестовой выборке против 93,7% без тонкой настройки), финальный сабмит сделан уже с изменением размера изображения со 150х150 на 240х240. Количество эпох на первых трех этапах увеличено до 10, последний этап увеличен до 15 эпох.

Изменим размер изображения:

In [98]:
H_IMG_SIZE = 240  # Горизонтальный размер изображения
V_IMG_SIZE = 240  # Вертикальный размер изображения

Для наглядности скопируем голову из пп 4.5., чтобы не искать ее выше:

In [99]:
head_new = Sequential([
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.25),
    Dense(256, activation='relu', bias_regularizer=l2(
        1e-3), activity_regularizer=l2(1e-4)),
    BatchNormalization(),
    Dense(CLASS_NUM, activation='softmax')
])

Добавим еще один параметр к аугментациям ImageDataAugmentor - a.RGBShift:

In [100]:
augment_module = a.Compose([
    # добавляем размытие по Гауссу и шум с вероятностью 7%
    a.Blur(p=0.07),
    a.GaussNoise(p=0.07),
    #  Установим параметры сдвига,поворота и масштабирования, а также укажем их вероятность.
    a.ShiftScaleRotate(shift_limit=0.08,
                       scale_limit=0.05,
                       border_mode=4,
                       rotate_limit=20,
                       p=0.7),

    a.RGBShift(),
    a.HueSaturationValue(),  # случайный оттенок и насыщенность
    a.HorizontalFlip(),

    a.Resize(V_IMG_SIZE, H_IMG_SIZE),
    # установим случайную яркость и контрастность изображений с вероятностью 50%

    a.OneOf([
            a.RandomBrightnessContrast(
                brightness_limit=0.3, contrast_limit=0.3),
            a.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)],
            p=0.3)
])

Теперь, используя созданный выше блок, запишем параметры для генератора:

In [101]:
# Подготовка параметров для аугментации:
all_datagen2 = ImageDataAugmentor(
    augment=augment_module,
    seed=RANDOM_SEED,
    validation_split=VAL_SPLIT)

Запустим генератор для того, чтобы получить аугментированные изображения:

In [102]:
# Сбросим генераторы
train_generator.reset()
val_generator.reset()

Запустим генераторы для аугментации изображений:

In [103]:
train_generator = all_datagen2.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training')

val_generator = all_datagen2.flow_from_directory(
    PATH+'train/',
    target_size=(V_IMG_SIZE, H_IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='validation')

Изменим коллбэки: так как модель обучается постепенно с разморозкой слоев, в процессе может возникать переобучение, поэтому EarlyStopping может прервать обучение до заверешния этапа обучения, что может негативно сказаться на всем процессе. Поэтому будем использовать только Checkpoint и ReduceLROnPlateau:

In [104]:
# Обновим коллбэки
callbacks_list2 = [checkpoint, lr_scheduler]

Загрузим базовую модель с весами ImageNet, укажем возможность обучения = False:

In [105]:
# Загрузим модель и установим trainable=False
base_model = EfficientNetB3(
    weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

Посмотрим количество слоев в базовой модели:

In [106]:
len(base_model.layers)

Соберем модель из base_model и head_new и проведем компиляцию:

In [107]:
# Собираем модель
model_efficientnetb3_ft = Sequential([
    base_model,
    head_new
])
# Проведем компиляцию с указанными параметрами
model_efficientnetb3_ft.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Теперь обучим модель:

In [108]:
# Обучение модели на тренировочной выброке и проверка валидационной точности на валидационной выборке
efficientnetb3_ft_fit = model_efficientnetb3_ft.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10,
    callbacks=callbacks_list2
)

Сохраним полученную модель и загрузим лучшие веса:

In [109]:
model_efficientnetb3_ft.save('../working/model_efficientnetb3_ft.hdf5')
model_efficientnetb3_ft.load_weights('best_model.hdf5')

Посмотрим точность для модели, базовая часть которой была заморожена и не обучалась:

In [110]:
scores = model_efficientnetb3_ft.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики train и val accuracy и loss:

In [111]:
acc = efficientnetb3_ft_fit.history['accuracy']
val_acc = efficientnetb3_ft_fit.history['val_accuracy']
loss = efficientnetb3_ft_fit.history['loss']
val_loss = efficientnetb3_ft_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Как мы видим, обучение достаточно хорошее, на всем промежутке метрики на валидационной выборке гораздо лучше, чем на обучающей.

Теперь разморозим половину слоев базовой части модели:

In [112]:
# Разморозим все слои модели
base_model.trainable = True
# Заморозим слои с 0 по len(base_model.layers)//2
for layer in base_model.layers[:len(base_model.layers)//2]:
    layer.trainable = False

Посмотрим, сколько переменных доступно для обучения:

In [113]:
len(base_model.trainable_variables)

Проведем компиляцию модели:

In [114]:
model_efficientnetb3_ft.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Проведем дообучение модели:

In [115]:
efficientnetb3_ft_fit = model_efficientnetb3_ft.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10,
    callbacks=callbacks_list2
)

Сохраним результат обучения и загрузим лучшие веса:

In [116]:
model_efficientnetb3_ft.save('../working/model_efficientnetb3_ft1.hdf5')
model_efficientnetb3_ft.load_weights('best_model.hdf5')

Проверим точность на валидационной выборке для модели, половина слоев которой была разморожена:

In [117]:
scores = model_efficientnetb3_ft.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики точности и функции потерь для обеих выборок:

In [118]:
acc = efficientnetb3_ft_fit.history['accuracy']
val_acc = efficientnetb3_ft_fit.history['val_accuracy']
loss = efficientnetb3_ft_fit.history['loss']
val_loss = efficientnetb3_ft_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Графики стали немного хуже, однако мы имеем хорошие показатели, уже близкие к тем, которые мы получали без тонкой настройки.

Теперь разморозим 3/4 слоев модели и проведем повторное обучение:

In [119]:
# Разморозим все слои модели
base_model.trainable = True
# Заморозим 1\4 нижних слоев модели
for layer in base_model.layers[:len(base_model.layers)//4]:
    layer.trainable = False

Посмотрим количество переменных, доступных для обучения:

In [120]:
len(base_model.trainable_variables)

Проведем компиляцию модели:

In [121]:
model_efficientnetb3_ft.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Проведем дообучение модели:

In [122]:
efficientnetb3_ft_fit = model_efficientnetb3_ft.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10,
    callbacks=callbacks_list2
)

Сохраним результат и загрузим лучшие веса:

In [123]:
model_efficientnetb3_ft.save('../working/model_efficientnetb3_ft2.hdf5')
model_efficientnetb3_ft.load_weights('best_model.hdf5')

Проверим точность на валидационной выборке для модели, 3/4 слоев которой разморожены:

In [124]:
scores = model_efficientnetb3_ft.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики точности и функции потерь для обеих выборок:

In [125]:
acc = efficientnetb3_ft_fit.history['accuracy']
val_acc = efficientnetb3_ft_fit.history['val_accuracy']
loss = efficientnetb3_ft_fit.history['loss']
val_loss = efficientnetb3_ft_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Качество графиков еще ухудшилось, но, возможно это связано с тем, что мы теперь наблюдаем маленькое изменение значений по оси y

Разморозим всю модель и проведем дообучение:

In [126]:
base_model.trainable = True

Проведем компиляцию модели:

In [127]:
model_efficientnetb3_ft.compile(loss="categorical_crossentropy", optimizer=optimizers.Adamax(
    learning_rate=LR), metrics=["accuracy"])

Проведем дообучение модели:

In [128]:
efficientnetb3_ft_fit = model_efficientnetb3_ft.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=15,
    callbacks=callbacks_list2
)

Сохраним результат и загрузим лучшие веса:

In [129]:
model_efficientnetb3_ft.save('../working/model_efficientnetb3_ft3.hdf5')
model_efficientnetb3_ft.load_weights('best_model.hdf5')

Проверим точность полученной нами модели:

In [130]:
scores = model_efficientnetb3_ft.evaluate_generator(
    val_generator, steps=len(val_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

Посмотрим графики функции точности и функции потерь для обеих выборок:

In [131]:
acc = efficientnetb3_ft_fit.history['accuracy']
val_acc = efficientnetb3_ft_fit.history['val_accuracy']
loss = efficientnetb3_ft_fit.history['loss']
val_loss = efficientnetb3_ft_fit.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

Финальным результатом получилось еще немного улучшить метрику точности и функции потерь. Графики ведут себя немного скачкообразно, однако мы так же наблюдаем эти изменения на маленьком диапазоне значений по y, поэтому они кажутся настолько ломаными.

## 6.2. Финальный Submit

Создадим и запустим генератор для аугментации изображений тестовой выборки:

In [132]:
test_datagen1 = ImageDataAugmentor()
submission_generator = test_datagen1.flow_from_dataframe(dataframe=sample_submission,
                                                         directory=PATH+'test_upload/',
                                                         x_col="Id",
                                                         y_col=None,
                                                         shuffle=False,
                                                         class_mode=None,
                                                         target_size=(
                                                             V_IMG_SIZE, H_IMG_SIZE),
                                                         batch_size=BATCH_SIZE)

Сделаем предсказания на тестовых данных, полученных из submission_generator:

In [133]:
predictions = model_efficientnetb3_ft.predict_generator(
    submission_generator, steps=len(submission_generator), verbose=1)
predictions = np.argmax(predictions, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

Создадим submission и запишем в него результаты предсказаний:

In [134]:
filenames = submission_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission_EfNetB3_Fine-Tune+size.csv', index=False)

# 7. Выводы по проекту

Данный проект позволил изучить различные параметры и модели сетей. В процессе выполнения приходилось использовать разные способы улучшения метрик, изучать параметры, думать как лучше модифицировать код.

Результатом выполнения проекта стал финальный сабмит с точностью:

Для дальнейшего улучшения метрик можно сделать следующее:
1. Увеличить размер train'а
2. Более тонко подойти к выбору параметров для аугментации изображений
3. Попробовать различные размеры изображений, которые допускает та или иная модель
4. Попробовать ансамблирование моделей
5. Провести более детальное исследование по различным способам аугментации и выбрать наиболее выгодный
6. Рассмотреть другие примеры сетей, пользуясь сайтом papperswithcode
7. Рассмотреть различные варианты head для каждой модели

В целом, выполнение проекта получилось объемным, но в процессе я изучил и запомнил много различной информации, необходимой для дальнейшей работы с нейронными сетями.
    Проект получилось сдать только после дедлайна, но это произошло в результате того, что я сильно увлекся с экспериментами и выработал весь лимит на использование GPU в неделю. Пришлось ждать еженедельного сброса, чтобы была возможность сохранить документ.